In [2]:
import numpy as np

#Set the numpy pseudo-random generator at a fixed value
#This helps with repeatable results everytime you run the code. 
np.random.seed(1000)

import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
import keras

In [3]:
os.environ['KERAS_BACKEND'] = 'tensorflow' # Added to set the backend as Tensorflow
#We can also set it to Theano if we want. 

In [4]:
#Iterate through all images in Parasitized folder, resize to 64 x 64
#Then save as numpy array with name 'dataset'
#Set the label to this as 0

image_directory = 'cell_images/'
SIZE = 64
dataset = []   
label = []  #We will add 0 to all parasitized images and 1 to uninfected.


In [5]:
#Iterate through all images in Parasitized folder, resize to 64 x 64
#Then save into the same numpy array 'dataset' with label 0
parasitized_images = os.listdir(image_directory + 'Parasitized/')
for i, image_name in enumerate(parasitized_images):    #Remember enumerate method adds a counter and returns the enumerate object
    
    if (image_name.split('.')[1] == 'png'):
        image = cv2.imread(image_directory + 'Parasitized/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(0)


In [6]:
#Iterate through all images in Uninfected folder, resize to 64 x 64
#Then save into the same numpy array 'dataset' but with label 1

uninfected_images = os.listdir(image_directory + 'Uninfected/')
for i, image_name in enumerate(uninfected_images):
    if (image_name.split('.')[1] == 'png'):
        image = cv2.imread(image_directory + 'Uninfected/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(1)

In [7]:
#Apply CNN
#Build the model
#conv and pool layers. with some normalization and drops in between.

INPUT_SHAPE = (SIZE, SIZE, 3)   #change to (SIZE, SIZE, 3)
inp = keras.layers.Input(shape=INPUT_SHAPE)

conv1 = keras.layers.Conv2D(32, kernel_size=(3, 3), 
                               activation='relu', padding='same')(inp)
pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
norm1 = keras.layers.BatchNormalization(axis = -1)(pool1)
drop1 = keras.layers.Dropout(rate=0.2)(norm1)
conv2 = keras.layers.Conv2D(32, kernel_size=(3, 3), 
                               activation='relu', padding='same')(drop1)
pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
norm2 = keras.layers.BatchNormalization(axis = -1)(pool2)
drop2 = keras.layers.Dropout(rate=0.2)(norm2)

flat = keras.layers.Flatten()(drop2)  #Flatten the matrix to get it ready for dense.

hidden1 = keras.layers.Dense(512, activation='relu')(flat)
norm3 = keras.layers.BatchNormalization(axis = -1)(hidden1)
drop3 = keras.layers.Dropout(rate=0.2)(norm3)
hidden2 = keras.layers.Dense(256, activation='relu')(drop3)
norm4 = keras.layers.BatchNormalization(axis = -1)(hidden2)
drop4 = keras.layers.Dropout(rate=0.2)(norm4)

out = keras.layers.Dense(2, activation='sigmoid')(drop4) 

model = keras.Model(inputs=inp, outputs=out)
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
print(model.summary())
 

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)       

In [8]:
#Split the dataset
#split the dataset into training and testing dataset.
# 1. Training data: 80%
# 2. Testing data: 20%

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(dataset, to_categorical(np.array(label)), test_size = 0.20, random_state = 0)

In [9]:
#Fit the model
history = model.fit(np.array(X_train), 
                         y_train, 
                         batch_size = 64, 
                         verbose = 1, 
                         epochs = 15,      
                         validation_split = 0.1,
                         shuffle = False
                     )

Epoch 1/15
311/311 [==============================] - 90s 290ms/step - loss: 0.4363 - accuracy: 0.7944 - val_loss: 1.1878 - val_accuracy: 0.7932
Epoch 2/15
311/311 [==============================] - 106s 340ms/step - loss: 0.2186 - accuracy: 0.9201 - val_loss: 0.1746 - val_accuracy: 0.9320
Epoch 3/15
311/311 [==============================] - 98s 314ms/step - loss: 0.1842 - accuracy: 0.9320 - val_loss: 0.4369 - val_accuracy: 0.7914
Epoch 4/15
311/311 [==============================] - 94s 301ms/step - loss: 0.1675 - accuracy: 0.9392 - val_loss: 0.2041 - val_accuracy: 0.9057
Epoch 5/15
311/311 [==============================] - 91s 293ms/step - loss: 0.1416 - accuracy: 0.9472 - val_loss: 0.1958 - val_accuracy: 0.9306
Epoch 6/15
311/311 [==============================] - 92s 296ms/step - loss: 0.1282 - accuracy: 0.9509 - val_loss: 0.1882 - val_accuracy: 0.9197
Epoch 7/15
311/311 [==============================] - 91s 291ms/step - loss: 0.1099 - accuracy: 0.9570 - val_loss: 0.1646 - val_a

In [10]:
#Accuracy calculation
# I'll now calculate the accuracy on the test data.

print("Test_Accuracy: {:.2f}%".format(model.evaluate(np.array(X_test), np.array(y_test))[1]*100))

173/173 [==============================] - 5s 28ms/step - loss: 0.1968 - accuracy: 0.9458
Test_Accuracy: 94.58%


In [12]:
#Save the model
model.save('malaria.h5')